In [ ]:
# Python 2
from __future__ import print_function
import numpy as np
np.set_printoptions(threshold=np.nan, linewidth=np.nan)

In [ ]:
import pickle
with open('df.pickle', mode='rb') as h:
    df = pickle.load(h)
with open('hex.pickle', mode='rb') as h:
    hex_data = pickle.load(h)
id_name = hex_data['id_name']

In [ ]:
with open('test.npy', mode='wb') as h:
    np.save(h, Y[0,:])

In [ ]:
import caffe  # os.environ['PYTHONPATH'].append('/home/libo/caffe/python')
import h5py
from os.path import join
img_dir = '../pascal12/JPEGImages'
for split in ['train', 'test']:
    filenames = df[df['split']==split]['img'].reset_index(drop=True)
    file_count = len(filenames)
    X = list()
    for i in range(0, file_count):
        X.append(caffe.io.load_image(join(img_dir, filenames[i])))
        progress = int(float(i) / file_count * 1000) / 10.0
        print('{} {}: {}%'.format(split, filenames[i], progress), end='\r')
    with open(split + '.npy', mode='wb') as h:  # mean pixle
        np.save(h, (sum(X) / file_count).mean(axis=(0, 1)))
    Y = np.zeros((len(X), len(id_name)), dtype=np.float16)
    pseudo_labels = df[df['split']==split]['pseudo_label'].reset_index(drop=True)
    for i in range(0, len(Y)):
        Y[i, pseudo_labels[i]] = 1
    with h5py.File('../{}.h5'.format(split), 'w') as h:
        h['X'] = np.array(X)  # TODO: exceeded 2GB numpy memory limit
        for i in range(0, len(id_name)):
            h[id_name[i]] = Y[:, i]
    with open('../{}.txt'.format(split), 'w') as h:
        h.write('{}.h5\n'.format(split))